## Auto-Read Datasets & PII Scan

Enviornment for developing a quick method to scan through datasets and see what PPID and PID are included.

In [1]:
from scipy import stats
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
loc1 = 'data/chicagoParole.csv'
loc2 = 'data/Strategic_Subject_List_-_Historical_20240320.csv'

## Original Function

In [3]:
def pii_violation(csv_filePath: str, pii_roots = ['Name', 'Date', 'Time', 'Address', 'Residence', 'County', 'State', 'District', 'Street', 'Code', 'Number', 'Age', 'Ethnicity', 'Gender', 'Occupation', 'Status', 'DOB', 'Year', 'Month', 'Day']):
    # read in the CSV
    data = pd.read_csv(csv_filePath)  

    # create a lowercase version of whatever was passed in or the default list
    pii_roots = [x.lower() for x in pii_roots] 

    # container for the hit columns
    suspected_pii = []

    # check each of the columns against our pii_roots (the default or a custom one)
    for col in data.columns:
        # make sure everything is the same regardless of casing
        lowercol = col.lower()

        # check the lower case versions, but append the normal column
        if len(set(lowercol.split(' ')).intersection(pii_roots)) > 0:
            # the roots matched, so the column(s) have a space as a delimiter
            suspected_pii.append(col)

        elif len(set(lowercol.split('_')).intersection(pii_roots)) > 0:
            # the roots matched, so the column(s) have '_' as a delimiter
            suspected_pii.append(col)
        elif len(set(lowercol.split('-')).intersection(pii_roots)) > 0:
            # the roots matched, so the column(s) have '-' as a delimiter
            suspected_pii.append(col)

    # hits in comparison to the columns
    hitRate = round(len(suspected_pii) / len(data.columns), 2)

    return suspected_pii, hitRate

## Class

Upgrades the function into something more robuts and useful.

In [15]:
class PIIScan():
    def __init__(self, filePath: str, pii = ['Name', 'Date', 'Time', 'Address', 'Street', 'Residence', 'Country', 'County', 'State', 'District', 'Code', 'Number', 'Age', 'Ethnicity', 'Gender', 'Occupation', 'Status', 'DOB', 'Year', 'Month', 'Day']):
        '''
        Reads a dataset (csv or xlsx supported) given a filepath, saves it and the metadata, and checks for partial or complete PII matches.

        Inputs:
            - (str) filePath: the relative or full path to the CSV or XLSX file
            - (list-like) pii: optional list of custom key words that we want to locate in the dataset features that may be PII. Default provided.

        Returns:
            - None. All information is saved to the object.

        Attributes Defined Here:
            - (str) filePath: The relative or full path to the data that was provided
            - (str) fileName: The name of the file minus the extension
            - (str) fileExtension: The extension name like 'csv' or 'xlsx'
            - (list: str) features: All column (feature) names
            - (list: str) roots: All key words that we assume are associated with PII (default provided)

        Attributes Defined Elsewhere:
            - (pd.DataFrame) df: Data stored in DataFrame format
            - (list: str) matches: List of unique column (feature) names that were found to be partial or complete matches to something in the PII list
            - (float) hitRate: Proportion of possibly PII columns in the dataset. Rounded to 2 decimal places)
            - (pd.Series) nan: Feature name and proportion of it's entries that are NAN
        '''
        
        # save the file path, name, and extension (mainly for report purposes)
        self.filePath = filePath
        self.fileName = Path(filePath).stem
        self.fileExtension = filePath.split('.')[-1]

        # get and save the data
        self._readFile()
        self.features = list(self.df.columns)

        # save the lowercase version of roots passed in or used from default
        self.roots = [root.lower() for root in pii]

        # check for PII hits
        print('Checking for PII Violations...')
        self._piiViolation()

        print('Reporting...\n')

        print(self)

    def __str__(self):
        return f' File: {self.fileName} \n File Type: {self.fileExtension} \n Features: {len(self.features)} \n Records: {len(self.df)} \n\n Possible PII Matches: {len(self.matches)} \n Hit Rate: {self.hitRate} \n\n Possible Matches: {self.matches} '

    def _readFile(self):
        '''
        Reads the datafile into a pandas DataFrame. Used in the initialization of the object.

        Inputs:
            - None

        Returns:
            - None

        Attributes Defined Here:
            - (pd.DataFrame) df: Data stored in DataFrame format
            - (pd.Series) nan: Feature name and proportion of it's entries that are NAN
        '''
        
        if self.fileExtension == 'csv':
            print('Reading CSV...')
        
            self.df = pd.read_csv(self.filePath)
        
        elif self.fileExtension == 'xlsx':
            print('Reading XLSX...')

            self.df = pd.read_excel(self.filePath)
        
        else:
            print(f'Extension not recognized: {self.fileExtension}')

        # Feature NAN analysis (what proportion of the feature is NAN?)
        print('Running feature NAN Analysis...')
        self.nan = self.df.isna().mean()
        
    def _piiViolation(self):
        '''
        Runs the PII-matching method. Given the pandas representation of the data, look at the columns (features) and record the ones that contain the PII keywords we defined.

        Inputs:
            - None

        Returns:
            - None

        Attributes Defined Here:
            - (list: str) matches: List of unique column (feature) names that were found to be partial or complete matches to something in the PII list
            - (float) hitRate: Proportion of possibly PII columns in the dataset. Rounded to 2 decimal places)
        '''
                
        # container for the hit columns
        suspected_pii = []

        # check each of the columns against our pii_roots (the default or a custom one)
        for col in self.features:
            # make sure everything is the same regardless of casing
            lowercol = col.lower()

            # check the lower case versions, but append the normal column
            if len(set(lowercol.split(' ')).intersection(self.roots)) > 0:
                # the roots matched, so the column(s) have a space as a delimiter
                suspected_pii.append(col)

            elif len(set(lowercol.split('_')).intersection(self.roots)) > 0:
                # the roots matched, so the column(s) have '_' as a delimiter
                suspected_pii.append(col)

            elif len(set(lowercol.split('-')).intersection(self.roots)) > 0:
                # the roots matched, so the column(s) have '-' as a delimiter
                suspected_pii.append(col)

        # get rid of any duplicates we've amassed and save it
        self.matches = list(np.unique(suspected_pii))
        self.hitRate = round(len(suspected_pii) / len(self.features), 2)

    def getData(self):
        '''
        Getter for the dataframe and the possible PII matches. 

        Inputs:
            - None

        Returns:
            - (pd.DataFrame) df: Data stored in DataFrame format
            - (list: str) matches: List of unique column (feature) names that were found to be partial or complete matches to something in the PII list
        '''
        return self.df, self.matches
    
    def getMatchSet(self):
        '''
        Getter for the auto-subset df of the possible matches.

        Inputs:
            - None

        Returns:
            - (pd.DataFrame): Subset data stored in DataFrame format
        '''
        return self.df[self.matches]
    
    def getMatchSet_Latex(self, rows = 5):
        '''
        Getter for the auto-subset df of the possible matches, but in latex format.

        Inputs:
            - None

        Returns:
            - (str): Latex tabular representation of the head of the data
        '''
        return print(self.df[self.matches].head(rows).to_latex(index = False))
    
    def getNan(self):
        '''
        Returns the columns that have some number of NANs in them.

        Inputs:
            - None

        Returns:
            - (pd.Series): Column (feature) names that have a proportion of NANs that greater than 0
        '''

        return self.nan[self.nan > 0]

## Testing the Class

In [16]:
# default roots
chicagoParole = PIIScan('data/chicagoParole.csv')
# print(chicagoParole)

Reading CSV...
Running feature NAN Analysis...
Checking for PII Violations...
Reporting...

 File: chicagoParole 
 File Type: csv 
 Features: 30 
 Records: 15630 

 Possible PII Matches: 12 
 Hit Rate: 0.4 

 Possible Matches: ['Age', 'County of Residence', 'Current Admission Date', 'Custody Date', 'Date of Birth', 'MSR/Parole Date', 'Name', 'Projected Discharge Date', 'Residence Zip Code', 'Sentence Date', 'Sentencing County', 'Veteran Status'] 


In [17]:
chicagoParole.getNan()

Projected Discharge Date    0.097057
Latitude                    0.001663
Longitude                   0.001663
New Offense Category        0.001599
Years Until Discharge       0.097057
dtype: float64

In [18]:
chicagoParole.getMatchSet().head()

,Age,County of Residence,Current Admission Date,Custody Date,Date of Birth,MSR/Parole Date,Name,Projected Discharge Date,Residence Zip Code,Sentence Date,Sentencing County,Veteran Status
0,71,Cook,1997-09-12 00:00:00,1995-03-18 00:00:00,1952-04-20 00:00:00,2022-08-05 00:00:00,"JONES, ROBERT",2025-08-05 00:00:00,60608,1997-09-08 00:00:00,Cook,No
1,69,Winnebago,2020-09-02 00:00:00,2020-02-20 00:00:00,1954-09-09 00:00:00,2021-02-19 00:00:00,"FOOTE, RAYMOND R.",2025-02-20 00:00:00,61102,2020-07-31 00:00:00,Winnebago,No
2,70,Cook,2000-04-04 00:00:00,1998-09-02 00:00:00,1953-11-06 00:00:00,2022-10-11 00:00:00,"ROGERS, CHARLES",2025-10-11 00:00:00,60445,2000-03-27 00:00:00,Cook,No
3,72,Cook,2019-01-11 00:00:00,2018-06-09 00:00:00,1951-04-24 00:00:00,2023-04-21 00:00:00,"FRENCH-SMITH, ALFRED",2026-04-23 00:00:00,60426,2019-01-09 00:00:00,Cook,No
4,68,Cook,2002-11-06 00:00:00,1982-04-09 00:00:00,1955-04-13 00:00:00,2021-11-30 00:00:00,"COVELLI, ROBERT",2024-11-30 00:00:00,60660,2002-10-11 00:00:00,Dupage,No


In [19]:
# print(chicagoParole.getMatchSet().head().to_latex())
chicagoParole.getMatchSet_Latex()

\begin{tabular}{rlllllllrlll}
\toprule
Age & County of Residence & Current Admission Date & Custody Date & Date of Birth & MSR/Parole Date & Name & Projected Discharge Date & Residence Zip Code & Sentence Date & Sentencing County & Veteran Status \\
\midrule
71 & Cook & 1997-09-12 00:00:00 & 1995-03-18 00:00:00 & 1952-04-20 00:00:00 & 2022-08-05 00:00:00 & JONES, ROBERT & 2025-08-05 00:00:00 & 60608 & 1997-09-08 00:00:00 & Cook & No \\
69 & Winnebago & 2020-09-02 00:00:00 & 2020-02-20 00:00:00 & 1954-09-09 00:00:00 & 2021-02-19 00:00:00 & FOOTE, RAYMOND R. & 2025-02-20 00:00:00 & 61102 & 2020-07-31 00:00:00 & Winnebago & No \\
70 & Cook & 2000-04-04 00:00:00 & 1998-09-02 00:00:00 & 1953-11-06 00:00:00 & 2022-10-11 00:00:00 & ROGERS, CHARLES & 2025-10-11 00:00:00 & 60445 & 2000-03-27 00:00:00 & Cook & No \\
72 & Cook & 2019-01-11 00:00:00 & 2018-06-09 00:00:00 & 1951-04-24 00:00:00 & 2023-04-21 00:00:00 & FRENCH-SMITH, ALFRED & 2026-04-23 00:00:00 & 60426 & 2019-01-09 00:00:00 & Cook & 

In [7]:
# chicago parole dataframe and possible matches
cpdf, cppm = chicagoParole.getData()

# commenting this line out because it shows real names
# cpdf[cppm].head()

In [8]:
other = PIIScan(loc2)
# print(other)

Reading CSV...


/var/folders/sw/s4dmzgv113lb3_sdwwd28tv40000gn/T/ipykernel_5442/2745220228.py:68: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.filePath)


Running feature NAN Analysis...
Checking for PII Violations...
Reporting...

 File: Strategic_Subject_List_-_Historical_20240320 
 File Type: csv 
 Features: 48 
 Records: 398684 

 Possible PII Matches: 15 
 Hit Rate: 0.31 

 Possible Matches: ['AGE CURR', 'AGE GROUP', 'AGE TO', 'IDOC RES STATE CODE', 'IDOC RES ZIP CODE', 'LATEST DATE', 'LATEST DOMESTIC ARR DATE', 'LATEST NARCOTIC ARR DATE', 'LATEST WEAPON ARR DATE', 'PREDICTOR RAT AGE AT LATEST ARREST', 'RACE CODE CD', 'SEX CODE CD', 'SSL LAST PTV DATE', 'STATUS I', 'TRAP STATUS'] 


In [9]:
# other dataframe and possible matches
otherdf, otherpm = other.getData()

# otherdf[otherpm]

In [10]:
other.getNan()

PREDICTOR RAT AGE AT LATEST ARREST    0.000256
AGE GROUP                             0.000384
AGE TO                                0.000384
STOP ORDER NO                         0.995844
PAROLEE I                             0.967182
MAJORITY DIST                         0.366240
DLST                                  0.852113
WEAPONS ARR CNT                       0.951974
LATEST WEAPON ARR DATE                0.951974
NARCOTICS ARR CNT                     0.767217
LATEST NARCOTIC ARR DATE              0.767217
IDOC RES CITY                         0.852156
IDOC RES STATE CODE                   0.852151
IDOC RES ZIP CODE                     0.853852
IDOC CPD DIST                         0.971852
DOMESTIC ARR CNT                      0.829123
LATEST DOMESTIC ARR DATE              0.829123
AGE CURR                              0.000604
SSL LAST PTV DATE                     0.964398
TRAP STATUS                           0.992179
TRAP FLAGS                            0.999737
SSL FLAGS    

In [11]:
largeParoleData = PIIScan('data/Parole pop subset.xlsx')

Reading XLSX...
Running feature NAN Analysis...
Checking for PII Violations...
Reporting...

 File: Parole pop subset 
 File Type: xlsx 
 Features: 30 
 Records: 15630 

 Possible PII Matches: 12 
 Hit Rate: 0.4 

 Possible Matches: ['Age', 'County of Residence', 'Current Admission Date', 'Custody Date', 'Date of Birth', 'MSR/Parole Date', 'Name', 'Projected Discharge Date', 'Residence Zip Code', 'Sentence Date', 'Sentencing County', 'Veteran Status'] 


In [12]:
largeParoleData.getNan()

Projected Discharge Date    0.097057
Latitude                    0.001663
Longitude                   0.001663
New Offense Category        0.001599
Years Until Discharge       0.097057
dtype: float64